## task01  知识图谱介绍
20210109 created    
这个notebook是参加 DataWhale 组织的<知识图谱组队学习>的学习笔记.      
原始的开源学习项目地址:     
https://github.com/datawhalechina/team-learning-nlp/tree/master/KnowledgeGraph_Basic

In [1]:
# step 1：导入 Neo4j 驱动包
from neo4j import GraphDatabase

In [2]:
# step 2：连接 Neo4j 图数据库
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "TA5HRyd7p6xfQUr"))

In [3]:
 # 添加 关系 函数
def add_friend(tx, name, friend_name):    
    tx.run("MERGE (a:Person {name: $name}) "          
           "MERGE (a)-[:KNOWS]->(friend:Person {name: $friend_name})",            
    name=name, friend_name=friend_name)

In [5]:
# 定义 关系函数
def print_friends(tx, name):
    for record in tx.run("MATCH (a:Person)-[:KNOWS]->(friend) WHERE a.name = $name "
                         "RETURN friend.name ORDER BY friend.name", name=name):
        print(record["friend.name"])

In [6]:
 # step 3：运行
with driver.session() as session:
    session.write_transaction(add_friend, "Arthur", "Guinevere")
    session.write_transaction(add_friend, "Arthur", "Lancelot")
    session.write_transaction(add_friend, "Arthur", "Merlin")
    session.read_transaction(print_friends, "Arthur")

Guinevere
Lancelot
Merlin


In [ ]:
6.2 py2neo模块：通过操作python变量，达到操作neo4j的目的


In [10]:
# step 1：导包
from py2neo import Graph, Node, Relationship

In [14]:
# step 2：构建图
g = Graph("http://localhost:7474", username="neo4j", password="TA5HRyd7p6xfQUr")
# 报错--ClientError: [Security.Unauthorized] The client is unauthorized due to authentication failure.

In [15]:
# step 3：创建节点
tx = g.begin()
a = Node("Person", name="Alice")
tx.create(a)
b = Node("Person", name="Bob")

In [16]:
# step 4：创建边
ab = Relationship(a, "KNOWS", b)

In [17]:
# step 5：运行
tx.create(ab)
tx.commit()

<Bookmark >

# 七、通过csv文件批量导入图数据

前面学习的是单个创建节点，不适合大批量导入。这里我们介绍使用neo4j-admin import命令导入，适合部署在docker环境下的neo4j。 其他导入方法也可以参考Neo4j之导入数据

csv分为两个nodes.csv和relations.csv，注意关系里的起始节点必须是在nodes.csv里能找到的：

  # nodes.csv需要指定唯一ID和nam,
  headers = [
  'unique_id:ID', # 图数据库中节点存储的唯一标识
  'name', # 节点展示的名称
  'node_type:LABEL', # 节点的类型，比如Person和Location
  'property' # 节点的其他属性
  ]

  # relations.csv
  headers = [
  'unique_id', # 图数据库中关系存储的唯一标识
  'begin_node_id:START_ID', # begin_node和end_node的值来自于nodes.csv中节点
  'end_node_id:END_ID',
  'begin_node_name',
  'end_node_name',
  'begin_node_type',
  'end_node_type',
  'relation_type:TYPE', # 关系的类型，比如Friends和Married
  'property' # 关系的其他属性
  ]

制作出两个csv后，通过以下步骤导入neo4j:

    两个文件nodes.csv ，relas.csv放在

  neo4j安装绝对路径/import

    导入到图数据库mygraph.db

  neo4j bin/neo4j-admin import --nodes=/var/lib/neo4j/import/nodes.csv --relationships=/var/lib/neo4j/import/relas.csv   --delimiter=^ --database=xinfang*.db

delimiter=^ 指的是csv的分隔符

    指定neo4j使用哪个数据库

  修改 /root/neo4j/conf/neo4j.conf 文件中的 dbms.default_database=mygraph.db

    重启neo4j就可以看到数据已经导入成功了
